# Modifies lattice before parameter files are applied

In [11]:
import numpy as np
from sirepo import lib
from rsbeams.rsdata.SDDS import readSDDS

Step 1: Just change element types
- RFCA -> RFCW 
- SBEN -> CSRCSBEND
- DRIF -> LSCDRIFT
- CONDITIONAL: IF LSCDRIFT is < d meters downstream of CSRBEND -> CSRDRIFT

Step 2: Edit element parameters

# Step 1

### loading

In [2]:
from sirepo import lib

# sirepo.lib cannot import just the lattice file so we put it in a dummy command file
# dummy command file can be run to generate a twi file we get element locations from
mref = lib.Importer('elegant').parse_file('run_file_dummy.ele')
m = mref.copy()

# Step 1: Convert Element Types

## 1. Convert Cavities

In [3]:
counter = 0 
for ele in m['models']['elements']:
    if ele['type'].lower() in ['rfca',]:
        ele['type'] = 'RFCW'
        counter += 1
counter

5

## 2. Convert all "drifts" to LSC drifts
This will not change the csrdrifts immediately after BC1

In [4]:
counter = 0 
for ele in m['models']['elements']:
    if ele['type'].lower() in ['drif', 'drift',]:
        ele['type'] = 'LSCDRIFT'
        counter += 1
counter

239

## 3. Convert all Dipoles to CSRCSBENDS
This will not change the csrdrifts immediately after BC1

In [5]:
counter = 0
for ele in m['models']['elements']:
    if ele['type'].lower() in ['sben', 'sbend',]:
        ele['type'] = 'CSRCSBEND'
        counter += 1
counter

10

## 3. Convert LSCDRIFTS to CSRDRIFTS Where Needed
This will not change the csrdrifts immediately after BC1

In [31]:
distance_threshold = 5.0

sdds_file = readSDDS('dummy.twi')
sdds_file.read()

In [56]:
DEBUG = False
def distance_to_closest(element, measure_to, sdds_file, element_instance=0, side='downstream'):
    element_names = sdds_file.columns['ElementName'][0]
    element_types = sdds_file.columns['ElementType'][0]
    element_positions = sdds_file.columns['s'][0]
    
    try:
        target_index = np.where(element == element_names)[0][element_instance]
    except IndexError:
        print(f"Element {element} was not found in beamline")
        return None

    target_position = element_positions[target_index]
    if DEBUG:
        print("target position", target_position)
    
    measurement_indices = np.where(measure_to == element_types)[0]
    measurement_positions = element_positions[measurement_indices]
    if DEBUG:
        print('measurement positions', measurement_positions)
    
    measurements = measurement_positions - target_position
    
    if side == 'upstream':
        # Distance to closest instance of `measure_to` upstream of element
        use_indices = np.where(measurements > 0.)
        if use_indices[0].size < 1:
            print('No measurements go upstream')
            return None
        return np.min(measurements[use_indices])
    elif side == 'downstream':
        # Distance to closest instance of `measure_to` upstream of element
        use_indices = measurements[np.where(measurements < 0.)]
        if use_indices[0].size < 1:
            print('No measurements go downstream')
            return None
        return np.abs(np.min())
    elif side == 'either':
        # Distance to closest instance of `measure_to` upstream of element
        return np.min(np.abs(measurements))
    else:
        raise ValueError('upstream, downstream or either')

In [59]:
counter = 0
for ele in m['models']['elements']:
    if ele['type'].lower() in ['lscdrift',]:
        distance_to_closest_dip = distance_to_closest(ele['name'], 'SBEN', sdds_file, side='upstream')
        if distance_to_closest_dip and distance_to_closest_dip < distance_threshold:
            print(ele['name'], distance_to_closest_dip)
            ele['type'] = 'CSRDRIFT'
            counter += 1
counter

Element D_ACM_ESD was not found in beamline
D_B106_X106 3.649459999999999
D_B107_X107 2.8371249999999986
D_B111_X111 1.299116999999999
D_B114_HV114 0.7388770000000005
D_B115_X115 0.7702700000000009
D_B116_D117 0.26468700000000034
Element D_B117_X117 was not found in beamline
Element D_B123_X124 was not found in beamline
Element D_B124_T124 was not found in beamline
Element D_B502_X502 was not found in beamline
D_B505_EID_X505_EID 2.778139999999979
Element D_B505_X505 was not found in beamline
D_B506_EID_X506_EID 1.6072399999999902
Element D_B506_X506 was not found in beamline
Element D_B507_X507 was not found in beamline
Element D_B508_X508 was not found in beamline
Element D_B512_X512 was not found in beamline
Element D_B513_X513 was not found in beamline
Element D_B514_X514 was not found in beamline
D_B601_Q601 4.819461999999987
D_B603_X603 1.6185419999999908
No measurements go upstream
No measurements go upstream
No measurements go upstream
No measurements go upstream
Element D_B612

54

# Step 2: Apply Element Parameters

## Settings for LSC up through BC1

In [73]:
# Common settings
high_freq_cutoff0 = -1
high_freq_cutoff1 = -1
lsc_bins = 1024

In [74]:
# cavity settings
rfcw_settings = {
    'N_KICKS': 100,
#     'SMOOTHING': ,  # Not sure if this is needed. Have not used it in past.
    'LSC': 1,
    'LSC_BINS': lsc_bins,
#     'LSC_LOW_FREQUENCY_CUTOFF0' ,  # Normally not needed
#     'LSC_LOW_FREQUENCY_CUTOFF1' ,  # Normally not needed
    'LSC_HIGH_FREQUENCY_CUTOFF0': high_freq_cutoff0,
    'LSC_HIGH_FREQUENCY_CUTOFF1': high_freq_cutoff1,
}

lscdrift_settings = {
#     'SMOOTHING': ,  # Not sure if this is needed. Have not used it in past.
    'LSC': 1,
    'BINS': lsc_bins,
#     'LOW_FREQUENCY_CUTOFF0' ,  # Normally not needed
#     'LOW_FREQUENCY_CUTOFF1' ,  # Normally not needed
    'HIGH_FREQUENCY_CUTOFF0': high_freq_cutoff0,
    'HIGH_FREQUENCY_CUTOFF1': high_freq_cutoff1,
}

csrcsbend_settings = {
    'N_SLICES': 25,
    'BINS': 1024,
    'SG_HALFWIDTH': 1
}

csrdrift_settings = {
    'DZ': 0.01,
    'USE_STUPAKOV': 1,
    'LSC_BINS': lsc_bins,
    'HIGH_FREQUENCY_CUTOFF0': high_freq_cutoff0,
    'HIGH_FREQUENCY_CUTOFF1': high_freq_cutoff1,
}

# MODIFY BEAMLINE PARAMETERS
Main beamline is  `INJ_BC1_EPD`

In [60]:
beamline_name = 'INJ_BC1_EPD'

In [61]:
target_beamline_index = None

In [63]:
for i, line in enumerate(m['models']['beamlines']):
    if line['name'].lower() == beamline_name.lower():
        target_beamline_index = i
        break
else:
    print("not found")

In [64]:
m['models']['beamlines'][target_beamline_index]

{'name': 'INJ_BC1_EPD', 'id': 547, 'items': [546, 537, 526]}

items in `m['models']['beamlines'][target_beamline_index]` refer to `id` numbers that might be in beamlines or elements needs to recursively go through all elements and modify

Should have done this before but I'll generate a mapping of index in beamlines and elements to `id` and `_id` respectively and inverse to prevent more looping lookups

In [65]:
beamline_map = {}  # m['models']['beamlines'] index to id
element_map = {} # m['models']['elements'] index to _id

In [66]:
for i,line in enumerate(m['models']['beamlines']):
    beamline_map[i] = line['id']

In [67]:
for i, element in enumerate(m['models']['elements']):
    element_map[i] = element['_id']

In [68]:
inv_beamline_map = {v:k for k,v in beamline_map.items()}
inv_element_map = {v:k for k,v in element_map.items()}

In [69]:
def edit_elements(model, beamline_index, lscdrift_modification=None, rcfw_modification=None, 
                  csrdrift_modification=None, csrcsbend_modification=None):
    # Hardcoded to edit LSC items here
    
    for comp in model['models']['beamlines'][beamline_index]['items']:
        # Is an element - check to edit
        if comp in inv_element_map:
            element = model['models']['elements'][inv_element_map[comp]]
            if element['type']  == 'RFCW':
                if rcfw_modification:
                    for k, v in rcfw_modification.items():
                        element[k.lower()] = v
                else:
                    print(element)
            elif element['type'] == 'LSCDRIFT':
                if lscdrift_modification:
                    for k, v in lscdrift_modification.items():
                        element[k.lower()] = v
                        element['cell_length'] = element['l']
                else:
                    print(element)
            elif element['type'] == 'CSRDRIFT':
                if csrdrift_modification:
                    for k, v in csrdrift_modification.items():
                        element[k.lower()] = v
                else:
                    print(element)
            elif element['type'] == 'CSRCSBEND':
                if csrcsbend_modification:
                    for k, v in csrcsbend_modification.items():
                        element[k.lower()] = v
                else:
                    print(element)

        elif comp in inv_beamline_map:
            new_index = inv_beamline_map[comp]
            edit_elements(model, new_index, 
                          lscdrift_modification=lscdrift_modification, rcfw_modification=rcfw_modification,
                          csrdrift_modification=csrdrift_modification, csrcsbend_modification=csrcsbend_modification)
            

In [75]:
# Edits elements
edit_elements(m, target_beamline_index, lscdrift_modification=lscdrift_settings, rcfw_modification=rfcw_settings,
              csrdrift_modification=csrdrift_settings, csrcsbend_modification=csrcsbend_settings)

In [76]:
# Check for edits
edit_elements(m, target_beamline_index, lscdrift_modification=None, rcfw_modification=None)

{'_id': 6, 'type': 'CSRDRIFT', 'name': 'D_MK_ELEGANT_BEGIN_HV106', 'l': 0.125917, 'order': 0.0, 'group': '', 'dz': 0.01, 'use_stupakov': 1, 'lsc_bins': 1024, 'high_frequency_cutoff0': -1, 'high_frequency_cutoff1': -1}
{'_id': 7, 'type': 'CSRDRIFT', 'name': 'D_HV106_GV106', 'l': 0.078596, 'order': 0.0, 'group': '', 'dz': 0.01, 'use_stupakov': 1, 'lsc_bins': 1024, 'high_frequency_cutoff0': -1, 'high_frequency_cutoff1': -1}
{'_id': 8, 'type': 'CSRDRIFT', 'name': 'D_GV106_B106', 'l': 0.126538, 'order': 0.0, 'group': '', 'dz': 0.01, 'use_stupakov': 1, 'lsc_bins': 1024, 'high_frequency_cutoff0': -1, 'high_frequency_cutoff1': -1}
{'_id': 9, 'type': 'CSRDRIFT', 'name': 'D_B106_X106', 'l': 0.078569, 'order': 0.0, 'group': '', 'dz': 0.01, 'use_stupakov': 1, 'lsc_bins': 1024, 'high_frequency_cutoff0': -1, 'high_frequency_cutoff1': -1}
{'_id': 10, 'type': 'CSRDRIFT', 'name': 'D_X106_Q106', 'l': 0.188727, 'order': 0.0, 'group': '', 'dz': 0.01, 'use_stupakov': 1, 'lsc_bins': 1024, 'high_frequency_cu

In [78]:
# Write out the model with LSC parameters set up to the end of Bunch Compressor
m.write_files('lattice_collective_effects')

{'commands': local('/home/vagrant/jupyter/research/fastfelo/FAST/ELEGANT/compression_study/dummy.ele'),
 'lattice': local('/home/vagrant/jupyter/research/fastfelo/FAST/ELEGANT/compression_study/injectortodump.lte'),
 'output_files': ['dummy.twi',
  'injector_BC1_cryo.GONI',
  'injector_BC1_cryo.X106',
  'injector_BC1_cryo.X107',
  'injector_BC1_cryo.X108',
  'injector_BC1_cryo.X109',
  'injector_BC1_cryo.X111',
  'injector_BC1_cryo.X113',
  'injector_BC1_cryo.X115',
  '%s.X117',
  'injector_BC1_cryo.X118',
  'injector_BC1_cryo.X120',
  'injector_BC1_cryo.X121',
  '%s.X124',
  'injector_BC1_cryo.X129',
  'cryotoFODOend.X440',
  'cryotoFODOend.X441',
  'cryotoFODOend.X444',
  'cryotoFODOend.X450',
  'cryotoFODOend.X455',
  'cryotoFODOend.X460',
  'cryotoFODOend.X465',
  'cryotoFODOend.X470',
  'cryotoFODOend.X475',
  'cryotoFODOend.X480',
  '%s.X502',
  'FODOend_todump.X502_EID',
  '%s.X505',
  'FODOend_todump.X505_EID',
  '%s.X506',
  'FODOend_todump.X506_EID',
  '%s.X507',
  '%s.X508',